In [1]:
import os
import pandas as pd
import matplotlib.pylab as plt

In [2]:
train = pd.read_csv('/Users/guoli/Desktop/kaggle/Porto/train.csv',usecols=['target'])
test = pd.DataFrame()

In [3]:
foldernames = os.popen('ls data').readlines()

In [4]:
for foldername in foldernames:
    foldername = foldername[:-1]
    train[foldername] = pd.read_csv('data/{0}/train4stack.csv'.format(foldername))
    #test[foldername] = pd.read_csv('data/{0}/test4stack.csv'.format(foldername))

In [5]:
train.head()

,target,changlin_20171108_1,changlin_20171108_2,guoli_20171108,guoli_20171109,mubi_20171109,tian_20171109
0,0,0.044521,0.046646,0.994861,0.883679,0.047776,0.265382
1,0,0.035425,0.033705,1.047057,0.618817,0.035362,0.256079
2,0,0.017495,0.021387,1.272133,0.641586,0.026082,0.252894
3,0,0.033988,0.014646,0.477906,0.542949,0.023179,0.251246
4,0,0.031818,0.031650,0.769130,0.565832,0.031547,0.248128


In [6]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
 
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_coefficient(preds,dtrain):
    y = dtrain.get_label()
    return 'gini', -gini_normalized(y,preds)

In [7]:
from sklearn.model_selection import KFold

class CustomGridCV(object):
    def __init__(self, X, y, model, metric, griddata, cv=5):
        self.X = X
        self.y = y
        self.model = model
        self.metric = metric
        self.params = self.gridpoints(griddata)
        self.cv = cv
        self.bestScore = None
        self.bestParams = None
        
    def gridpoints(self, data):
        newparams = [{}]
        for k in data.keys():
            params = newparams
            newparams = []
            for v in data[k]:
                for param in params:
                    item = param.copy()
                    item[k]=v
                    newparams.append(item)           
        return newparams
    
    def GridSearch(self):
        for param in self.params:
            self.model.set_params(**param)
            score = self.KFoldScore()
            if self.bestScore==None or self.bestScore<score:
                self.bestScore = score
                self.bestParams = param
            print("Score: {0:.5f}, Params: {1}".format(score,param))
    
    def KFoldScore(self):
        kf = KFold(n_splits=5, shuffle=True, random_state=2)
        y_pred = np.zeros(len(self.y))

        for train_index, test_index in kf.split(self.X):
            train_X, test_X = self.X[train_index], self.X[test_index]
            train_y, test_y = self.y[train_index], self.y[test_index]
            self.model.fit(train_X,train_y)
            y_pred[test_index] = self.model.predict(test_X)

        return self.metric(self.y,y_pred)
    
    def Best(self):
        return self.bestScore, self.bestParams

In [8]:
import pandas as pd
import numpy as np

def PrepareData(df):
    #df = pd.read_csv(trainpath,nrows=nrows)

    features = list(df.columns)
    target = 'target'
    features.remove(target)

    X = np.array(df[features])
    y = np.array(df[target])
    return X, y

In [9]:
X, y = PrepareData(train)

In [10]:
from xgboost import XGBRegressor

# model = XGBRegressor(max_depth=3)
# griddata = {'n_estimators':[20,50,100,200,300]}
# GCV = CustomGridCV(X, y, model, gini_normalized, griddata)
# GCV.GridSearch()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

model = XGBRegressor(max_depth=2, 
                     learning_rate=0.005, 
                     n_estimators=1000, 
                     objective='reg:linear', 
                     gamma=2, 
                     min_child_weight=2, 
                     subsample=0.7, 
                     colsample_bytree=0.7)


model.fit(X_train,y_train,
          eval_set=[(X_train,y_train),(X_test,y_test)],
          eval_metric=gini_coefficient, 
          early_stopping_rounds=10,
          verbose=10)

[0]	validation_0-rmse:0.497851	validation_1-rmse:0.497845	validation_0-gini:-0.260005	validation_1-gini:-0.252897
Multiple eval metrics have been passed: 'validation_1-gini' will be used for early stopping.

Will train until validation_1-gini hasn't improved in 10 rounds.
[10]	validation_0-rmse:0.477031	validation_1-rmse:0.476975	validation_0-gini:-0.279608	validation_1-gini:-0.271875
[20]	validation_0-rmse:0.45739	validation_1-rmse:0.457279	validation_0-gini:-0.281515	validation_1-gini:-0.273106
[30]	validation_0-rmse:0.438862	validation_1-rmse:0.438696	validation_0-gini:-0.28199	validation_1-gini:-0.273654
[40]	validation_0-rmse:0.421397	validation_1-rmse:0.421173	validation_0-gini:-0.282417	validation_1-gini:-0.273997
[50]	validation_0-rmse:0.404945	validation_1-rmse:0.404661	validation_0-gini:-0.283659	validation_1-gini:-0.275264


In [30]:
print model.feature_importances_

[ 0.18438539  0.26744187  0.12790698  0.24418604  0.17607974]


In [75]:
print gini_normalized(y,model.predict(X))

0.434046400807


In [68]:
#model.predict_proba(np.array(test))[:,1]

In [144]:
result = pd.read_csv('/Users/guoli/Desktop/kaggle/Porto/test.csv',usecols=['id'])
result['target'] = model.predict_proba(np.array(test))[:,1]

In [146]:
result.head()

,id,target
0,0,0.026132
1,1,0.025951
2,2,0.025694
3,3,0.021864
4,4,0.029668


In [147]:
result.to_csv('xgb_nn_encoder.csv',index=False)

In [148]:
! head xgb_nn_encoder.csv

id,target
0,0.0261317432156
1,0.0259512759203
2,0.0256943898781
3,0.021864325779
4,0.0296683108862
5,0.0333643762681
6,0.0218926572291
8,0.0292410953604
10,0.0391363268473
